In [66]:
import os
from math import log10
import math

In [67]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

### Tokenization

In [68]:
par_path = "data"
dic = []
# Put all the document in the same list
for sub in os.listdir(par_path):
    file_path = os.path.join(par_path, sub)
    if os.path.isfile(file_path) and sub.endswith(".txt"):
        with open(file_path, 'r') as f:
            content = f.read()
            dic.append(content)

In [69]:
# seperate every words in each documents 
sep_dic = []
for content in dic:
    content = content.split()
    sep_dic.append(content)

In [70]:
punc = '''!()-[]{};:"\,<>./?@#$%^&*_~` '''
punc_dic = []
for con in sep_dic:
    new_con = []
    for term in con:
        new_t = term.replace('!', '').replace('(', '').replace(')', '').replace('-', '').replace('[', '').replace(']', '').replace('{', '').replace('}', '').replace(';', '').replace(':', '').replace('"', '').replace('\'', '').replace(',', '').replace('<', '').replace('>', '').replace('.', '').replace('/', '').replace('?', '').replace('@', '').replace('#', '').replace('$', '').replace('%', '').replace('^', '').replace('&', '').replace('*', '').replace('*', '').replace('_', '').replace('~', '').replace('`', '').replace(' ', '')
        new_con.append(new_t)
    punc_dic.append(new_con)

punc = '''!()-[]{};:"\,<>./?@#$%^&*_~` '''
punc_dic = []
for con in sep_dic:
    new_con = []
    for term in con:
        for ele in punc:
            new_t = term.replace(ele, '')
            new_con.append(new_t)
    punc_dic.append(new_con)

In [71]:
# Remove numeric and null term
oth_dic = []
for con in punc_dic:
    temp_con = []
    for term in con:
        if any(char.isnumeric() for char in term):
            continue
        if len(term) > 0:
            temp_con.append(term)
    oth_dic.append(temp_con)

### Lowercase

In [72]:
low_dic = []
for con in oth_dic:
    temp_con = []
    for term in con:
        term = term.lower()
        temp_con.append(term)
    low_dic.append(temp_con)

### Stemming using Porter's algo

In [73]:
stem_dic = []
for con in low_dic:
    temp_con = []
    for term in con:
        new_t = ps.stem(term)
        temp_con.append(new_t)
    stem_dic.append(temp_con)

### Preprocessing the stopwords

In [74]:
with open('stopwords.txt') as st:
    stopword = list(st)

In [75]:
stopwords = [words.split() for words in stopword]
# put every stopword in different list into the same list
stopwords = sum(stopwords, []) 

### Remove stopwords

In [76]:
stop_dic = []
for con in stem_dic:
    temp_con = []
    for term in con:
        if term not in stopwords or term.isnumeric() or len(term) == 0:
            temp_con.append(term)
    stop_dic.append(temp_con)

### Term frquency

In [77]:
tf = []
for con in stop_dic:
    temp = {}
    for term in con:
        if term not in temp:
            temp[term] = 1
        else:
            temp[term] = temp[term] + 1
    tf.append(temp)

### Dictionary

In [78]:
dic = []
for con in stop_dic:
    for term in con:
        if term not in dic:
            dic.append(term)

In [79]:
with open("output/dictionary.txt", "w") as output_file:
    for content in dic:
        output_file.write(content+ '\n')

### Document frequency

In [80]:
df = {}
for term in dic:
    df[term] = 0

In [81]:
for term in dic:
    for con in stop_dic:
        if term in con:
            df[term] = df[term] + 1

### IDF

In [82]:
idf = {}
N = 1095
for term in dic:
    df_term = df[term]
    idf[term] = math.log10(N/df_term)

### Tf-Idf

In [83]:
tfidf = []
for con in stop_dic:
    doc = {}
    for term in con:
        doc[dic.index(term)] = tf[stop_dic.index(con)][term] * idf[term]
    tfidf.append(doc)

In [84]:
item = []
for doc in tfidf:
    temp = 0
    for term in doc.items():
        temp += 1
    item.append(temp)

In [85]:
doc_id = 0
for doc in tfidf:
    doc_id += 1
    with open(f"output/{doc_id}.txt", "w") as output_file:
        output_file.write(str(item[doc_id-1]) + "\n")
        for term_idx, tfidf_s in doc.items():
            output_file.write(f"{term_idx}: {tfidf_s}\n")

### Cosine similarity

In [86]:
doc1 = tfidf[0]
doc2 = tfidf[1]

In [87]:
def cosine_similarity(doc_vec1, doc_vec2):
    dot_product = sum(doc1[term] * doc2[term] for term in doc1 if term in doc2)
    
    magnitude1 = math.sqrt(sum(doc1[term] ** 2 for term in doc1))
    magnitude2 = math.sqrt(sum(doc2[term] ** 2 for term in doc2))
    
    if magnitude1 == 0 or magnitude2 == 0:
        return 0
    else:
        cos_sim = dot_product / (magnitude1 * magnitude2)
        return cos_sim

In [88]:
sim12 = cosine_similarity(doc1, doc2)
print(sim12)

0.2712112524896092
